In [1]:
import sys
import mosek
import numpy.random as rdm

In [2]:
inf = 0.0

In [3]:
def streamprinter(text):
    sys.stdout.write(text)
    sys.stdout.flush()

In [4]:
def calculate(m, n, c, mu, nu):
    with mosek.Env() as env:
        with env.Task(0,0) as task:
            task.set_Stream(mosek.streamtype.log,streamprinter)
            bkc = [mosek.boundkey.fx] * (m + n)
            bkx = [mosek.boundkey.lo] * m * n
            blc = [i for i in mu] + [i for i in nu]
            buc = [i for i in mu] + [i for i in nu]
            blx = [0] * m * n
            bux = [+inf] * m * n
            asub = [[int(i / n), m + (i % n)] for i in range(m * n)]
            aval = [[1, 1] for i in range(m * n)]
            numvar = len(bkx)
            numcon = len(bkc)
            task.appendvars(numvar)
            task.appendcons(numcon)
            for j in range(numvar):
                task.putcj(j, c[j])
                task.putvarbound(j, bkx[j], blx[j], bux[j])
                task.putacol(j, asub[j], aval[j])
            for i in range(numcon):
                task.putconbound(i, bkc[i], blc[i], buc[i])
            task.putobjsense(mosek.objsense.minimize)
            task.optimize()
            task.solutionsummary(mosek.streamtype.msg)
            xx = [0.] * numvar
            task.getxx(mosek.soltype.bas, xx)
            return xx

In [5]:
def main():
    #Give the size of the problem
    m = 1000
    n = 1000
    #Generate random value for c, mu, nu
    c1 = rdm.rand(m, n)
    c = c1.reshape(m * n)
    total = 10
    mu1 = rdm.rand(m)
    nu1 = rdm.rand(n)
    mu = [i * total / sum(mu1) for i in mu1]
    nu = [i * total / sum(nu1) for i in nu1]
    #Solve LP problem for optimal transport
    xx = calculate(m, n, c, mu, nu)
    #Write the solution into a TXT file
    file = open('solution.txt', 'w')
    for i in range(m * n):
        #print('x[' + str(int(i/n)) + '][' + str(i%n) + ']=' + str(xx[i]))
        file.write(str('%.2f' % xx[i])+" ")
        if (i + 1) % n == 0:
            file.write('\n')
    file.close()

In [10]:
if __name__ == "__main__":
    main()

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 2000            
  Cones                  : 0               
  Scalar variables       : 1000000         
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator - tries                  : 0                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.06            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.92    
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 2000            
  Cones                  :